In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

from pytorch_lightning.trainer import Trainer
from model_interface import MInterface
from data_interface import DInterface


In [2]:
args = {
    'res_dir': './results',
    'ex_name': 'debug',
    'check_val_every_n_epoch': 1,
    'dataset': 'PTM',
    'model_name': 'MeToken',
    'lr': 1e-4,
    'lr_scheduler': 'onecycle',
    'offline': 1,
    'seed': 114514,
    'batch_size': 16,
    'num_workers': 16,
    'pad': 1024,
    'min_length': 40,
    'path': './data_test/generalization/qPTM_dataset/',
    'with_null_ptm': 0,
    'epoch': 20,
    'augment_eps': 0.0,
    'module_type': 94,
    'weight_type': 0,
    'gamma': 2.0,
    'final_tau': 1e-4,
    'pretrain': 0,
    'test_only': 1,
    'inference_pos':None,
    'ckpt_from_deepspeed': 0,
    'ckpt_path': "pretrained_model/checkpoint.ckpt",
    'gpus': [0],
    'strategy': 'auto',
    'wandb_offline': 1
}

In [3]:
data_module = DInterface(**args)
data_module.setup(stage="test")

In [4]:
import json


trainer_config = {
    'gpus': args['gpus'],
    'strategy': args['strategy'],
    'accelerator': 'gpu',
    'resume_from_checkpoint': args['ckpt_path']
}

trainer = Trainer(**trainer_config)
model = MInterface.load_from_checkpoint(trainer_config["resume_from_checkpoint"], strict=False,model_name=args["model_name"])
trainer.test(model,data_module)
metrics = model.cal_metric(path=args["path"])
with open(os.path.join(args["res_dir"], args["ex_name"], 'metrics.json'), 'w') as file_obj:
    json.dump(metrics, file_obj)

/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_zero_deprecation(
/root/anaconda3/envs/dyMEAN/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   7%|▋         | 2/27 [00:03<00:43,  1.75s/it]accuracy: 0.9500, precision: 0.9500, recall: 0.9167, f1 score: 0.9222, mcc score: 0.9167, auroc: 0.7890, auprc: 0.7633
